Модуль перевода

In [ ]:
from openai import OpenAI, AsyncOpenAI
import sqlite3
import dotenv
import os
import asyncio

In [ ]:
PROMPT = """Перевод квестов GregTech New Horizons на русский язык
GregTech New Horizons - это модификация для игры Minecraft, которая добавляет множество новых квестов и механик
ты мне присылаешь перевод в этом же формате
Символы § и % нужно сохранять
Перевод должен учитывать специфику игры и не переводить лишнее, а также можно использовать англицизмы, сохранять английские названия предметов, если уместно
Будь дружелюбен, сохраняй стиль и тон оригинала, проводи адаптацию, а не просто переводи слово в слово
В ответ я хочу получить только лучший перевод
Названия модов не переводи
Используй общепринятые термины и названия, если они есть, например:
End Stone = Эндерняк
Heavy Duty Plates = Сверхпрочное покрытие
macerate = дробить
Void protection = Защита от переполнения
Recipe locking = Блокировка рецепта
Americium doped Wafer = Америцием легированная вафля
Phosphorus doped Wafer = Вафля легированная фосфором
Cobaltite = Кобальтин
Tier = уровень
§split§ - системная метка для разделения текста, сохраняй ее неизменной"""

In [ ]:
if not dotenv.load_dotenv('.env'):
    print('No .env file found')
    exit(1)
OPENAI_ENDPOINT = os.getenv('OPENAI_ENDPOINT')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
MODEL_ID = os.getenv('MODEL_ID')
if not OPENAI_ENDPOINT or not OPENAI_API_KEY or not MODEL_ID:
    print('Missing environment variables')
    exit(1)

In [ ]:
def get_to_translate() -> dict:
    conn = sqlite3.connect('translate.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute('SELECT id, source_title, source_desc FROM translate WHERE target_title IS NULL LIMIT 1')
    row = c.fetchone()
    conn.close()
    if row:
        return dict(row)
    return {}

def get_to_translate_many() -> list[dict]:
    conn = sqlite3.connect('translate.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute('SELECT id, source_title, source_desc FROM translate WHERE target_title IS NULL')
    rows = c.fetchall()
    conn.close()
    return [dict(row) for row in rows]

In [ ]:
def update_translation(id: str, target_title: str, target_text: str):
    conn = sqlite3.connect('translate.db')
    c = conn.cursor()
    c.execute('UPDATE translate SET target_title = ?, target_desc = ? WHERE id = ?', (target_title, target_text, id))
    conn.commit()
    conn.close()

In [ ]:
def translate(title: str, text: str) -> tuple[str, str]:
    to_translate = '§split§'.join([title, text])
    client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_ENDPOINT)
    completion = client.chat.completions.create(model=MODEL_ID,
                                   messages=[
                                       {'role': 'system', 'content': PROMPT},
                                       {'role': 'user', 'content': to_translate}
                                   ])

    splited = completion.choices[0].message.content.split('§split§', 1)
    if len(splited) == 1:
        print(completion.choices[0].message.content)
        exit(1)
    return completion.choices[0].message.content.split('§split§', 1)

async def translate_async(title: str, text: str, semaphore: asyncio.Semaphore) -> tuple[str, str]:
    async with semaphore:
        to_translate = '§split§'.join([title, text])
        client = AsyncOpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_ENDPOINT)
        completion = await client.chat.completions.create(model=MODEL_ID,
                                       messages=[
                                           {'role': 'system', 'content': PROMPT},
                                           {'role': 'user', 'content': to_translate}
                                       ])
        splited = completion.choices[0].message.content.split('§split§', 1)
        if len(splited) == 1:
            print(completion.choices[0].message.content)
            exit(1)
        return completion.choices[0].message.content.split('§split§', 1)

In [ ]:
def single_thread_main():
    for i in range(4500):
        row = get_to_translate()
        if not row:
            print('Nothing to translate')
            exit(0)
        
        id = row['id']
        source_title = row['source_title']
        source_desc = row['source_desc']
        
        print(f'Translating {i}: {source_title}')
        target_title, target_text = translate(source_title, source_desc)
        
        update_translation(id, target_title, target_text)

In [ ]:
async def translate_and_save(one_row: dict, semaphore: asyncio.Semaphore):
    id = one_row['id']
    source_title = one_row['source_title']
    source_desc = one_row['source_desc']

    target_title, target_text = await translate_async(source_title, source_desc, semaphore)
    print(f'Translated {id}: {source_title}')

    update_translation(id, target_title, target_text)

async def async_main():
    import asyncio
    rows = get_to_translate_many()
    if not rows:
        print('Nothing to translate')
        exit(0)

    semaphore = asyncio.Semaphore(7)
    tasks = [translate_and_save(row, semaphore) for row in rows]
    await asyncio.gather(*tasks)

await async_main()
print('All done')